In [1]:
%load_ext lab_black
%cd ../../..

/home/shim/shim/dev/log-analytics


In [2]:
import torch
import numpy as np
import pandas as pd
import pickle

In [3]:
df = pd.read_csv("data/ori/test.csv")

In [4]:
deck1 = torch.load("results/distilbert-base-uncased-ver7/distilbert-base-uncased-focal-AdamW-lr1e-05-ver7-os10_3-deck1.pth")
deck2 = torch.load("results/distilbert-base-uncased-ver7/distilbert-base-uncased-focal-AdamW-lr1e-05-ver7-os10_3-deck2.pth")

In [5]:
deck1["feat"].shape, deck2["feat"].shape

(torch.Size([421079, 768]), torch.Size([1095951, 768]))

In [6]:
deck1["feat_"] = deck1["feat"].cuda()
deck2["feat_"] = deck2["feat"].cuda()

In [23]:
deck1.keys(), deck2.keys()

(dict_keys(['feat', 'otext', 'tlevel', 'fclevel', 'feat_']),
 dict_keys(['feat', 'otext', 'fclevel', 'ids', 'feat_']))

In [7]:
j = 0
shown = False

In [8]:
dist_ = torch.norm(deck1["feat_"] - deck2["feat_"][j, None], p=None, dim=1)
values_, indices_ = dist_.topk(4, largest=False)
tlevels = deck1["tlevel"][indices_]
print("values:", values_)
print("indices:", indices_)
print("id:", deck1["tlevel"][indices_])
j += 1

values: tensor([0.0031, 0.0031, 0.0031, 0.0031], device='cuda:0')
indices: tensor([17556, 17365, 17671, 19175], device='cuda:0')
id: tensor([0, 0, 0, 0])


In [21]:
while shown or (deck1["tlevel"][indices_[0]] in [0, 1, 3, 5] and values_[0] < 0.5):
    # while shown or (deck1["tlevel"][indices_[0]] in [0, 1, 3, 5]):
    shown = False

    dist_ = torch.norm(deck1["feat_"] - deck2["feat_"][j, None], p=None, dim=1)
    values_, indices_ = dist_.topk(4, largest=False)
    tlevels = deck1["tlevel"][indices_]
    # print("values:", values)
    # print("indices:", indices)
    # print("id:", deck1["tlevel"][indices])
    j += 1

In [22]:
print("*  Submit :", df.full_log[deck2["ids"][j - 1][0] - 1000000])
print("----------------------------------------------------------------------------------------")
print("*   index :", j - 1)
print("*    dist :", " ".join(map(lambda x: f"{x:.4f}", values_.cpu().tolist())))
print("* fc-level:", deck2["fclevel"][j - 1])
print("----------------------------------------------------------------------------------------")

for idx in indices_:
    print("* Level", deck1["tlevel"][idx].item(), ":", deck1["otext"][idx])
    print("----------------------------------------------------------------------------------------")

shown = True

*  Submit : type=ANOM_ABEND msg=audit(1600394668.751:3360): auid=4294967295 uid=48 gid=48 ses=4294967295 subj=system_u:system_r:httpd_t:s0 pid=6914 comm="httpd" reason="memory violation" sig=11
----------------------------------------------------------------------------------------
*   index : 11440
*    dist : 0.9715 1.1397 1.1826 1.3108
* fc-level: tensor(1)
----------------------------------------------------------------------------------------
* Level 1 : type=USER_AVC msg=audit(1613363937.137:2481): pid=813 uid=81 auid=4294967295 ses=4294967295 subj=system_u:system_r:system_dbusd_t:s0-s0:c0.c1023 msg='avc: denied { send_msg } for msgtype=method_call interface=org.freedesktop.login1.Manager member=CreateSession dest=org.freedesktop.login1 spid=30614 tpid=860 scontext=system_u:system_r:httpd_t:s0 tcontext=system_u:system_r:systemd_logind_t:s0 tclass=dbus exe="/usr/bin/dbus-daemon" sauid=81 hostname=? addr=? terminal=?'
----------------------------------------------------------------

---

4개가 다 일치할 때는 level별로 threshold를 따로 두는 방식으로 하고
* level0 - 좀 낮게 (비교군이 많기 때문에) - 0.8?
* level1 - 전처리를 없애니까 dist가 증가했음. 0.8?
* level2 - 
* level3 - 꽤 높게 (변화무쌍하니까) - 1.0?
* level4 - 
* level5 - 꽤 높게 (변화무쌍하니까) - 1.0
* level6 - 나름 변화가 큼 - 0.8
* 섞인 경우 - 비슷한 케이스가 별로 없었다는 뜻이니까 낮게 - 0.2

2, 4, 6에 대해서는 아직 데이터가 더 모여야 하겠다

level0 길이가 긴거는 맞아도 dist가 높게(0.3이상) 나오고, 길이가 짧은건 틀려도 dist가 탖게 나오는 것 같은데...  
길이도 입력 인수로 해야할까?

※ TODO: 전체에 대해 dist 구한 다음에 각 level/군집별로 top dist 분포가 어떻게 나타나는지 그래프를 그려보자

In [24]:
deck1.keys()

dict_keys(['feat', 'otext', 'tlevel', 'fclevel', 'feat_'])

In [31]:
(deck1["tlevel"] == 6).nonzero()

tensor([[101108],
        [175083],
        [212017],
        [219277],
        [220791],
        [334090],
        [369633],
        [387381]])

In [35]:
for i in (deck1["tlevel"] == 6).nonzero()[:, 0]:
    print(deck1["otext"][i])

sshd: Bad protocol version identification '\003' from 78.128.113.18 port 1073
kernel: device virbr0-nic entered promiscuous mode
kernel: device enp2s0 entered promiscuous mode
useradd: new group: name=test, GID=1001
sshd: error: maximum authentication attempts exceeded for root from 192.168.0.197 port 61153 ssh2 [preauth]
kernel: device virbr0-nic entered promiscuous mode
kernel: device virbr0-nic entered promiscuous mode
kernel: device enp2s0 entered promiscuous mode
